## FILE TO EXPLORE

/home/mario/Dropbox/PycharmProjects/ndsgo/data_processing


In [2]:
import sys

print(sys.path)

#add '/home/mario/Dropbox/PycharmProject/ndsgo/' to path
sys.path.append('/home/mario/Dropbox/PycharmProjects/ndsgo/')
#print(sys.path)

['', '/home/mario/anaconda3/envs/tensorenv2/lib/python27.zip', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/plat-linux2', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/lib-tk', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/lib-old', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/lib-dynload', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/site-packages', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/site-packages/Sphinx-1.4.6-py2.7.egg', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/site-packages/IPython/extensions', '/home/mario/.ipython']


## READ PROPERTIES FILE

In [18]:
with open(filename+'.prop','r') as f:
    
    class Properties():
        pass
    properties=Properties
    properties.ind_num_examples=[int(c) for c in f.readline().strip().split(',')]
    properties.train_num_examples=sum(properties.ind_num_examples[:-1])
    properties.test_num_examples=int(properties.ind_num_examples[-1])
    properties.total_num_examples=properties.train_num_examples+properties.test_num_examples
    properties.num_channels=int(f.readline().strip())
    properties.boardsize=int(f.readline().strip())
print(properties.ind_num_examples,properties.total_num_examples,properties.train_num_examples,
      properties.test_num_examples,properties.num_channels,properties.boardsize)

([771385, 771385, 771385, 771385, 385688, 385692], 3856920, 3471228, 385692, 4, 19)


### TRAIN AND TEST ATTRIBUTES AND SIZES

In [19]:
import gzip
import struct
    
        
total_images=0
total_bytes=0
for s in ['train','test']:   
    with gzip.open(filename+'-'+s+'-labels-idx1-ubyte.gz') as bytestream:
        magic_number=bytestream.read(4)
        data=bytestream.read()
        bytes_label=len(data)
    
    with gzip.open(filename+'-'+s+'-vectors-idx4-ubyte.gz') as bytestream:
        magic_number=bytestream.read(4)
        num_images=struct.unpack('>I',bytestream.read(4))[0]
        size_1=struct.unpack('>I',bytestream.read(4))[0]
        size_2=struct.unpack('>I',bytestream.read(4))[0]
        size_3=struct.unpack('>I',bytestream.read(4))[0]
        data=bytestream.read()
        bytes_vector=len(data)
    total_images+=num_images
    total_bytes+=bytes_label+bytes_vector+24
    print('{}-labels. bytes_label:{}'.format(s,bytes_label))
    print('{} BYTES. bytes_vector:{} bytes_label:{}'.format(s.upper(),bytes_vector,bytes_label))
    print(s,num_images,size_1,size_2,size_3,bytes_label+bytes_vector,float(bytes_vector)/(size_1*size_2*size_3))
    
    if num_images==bytes_vector/(size_1*size_2*size_3):
        print('{} CORRECT'.format(s))
    else:
        print('{} INCORRECT'.format(s))
print('total_images',total_images,'total_bytes',total_bytes)


with open(filename + '.bin','rb') as bytestream:
    data=bytestream.read()
    bin_total_bytes=len(data)
    bin_total_images=len(data)/(size_1*size_2*size_3+2)
    print('bin',size_1,size_2,size_3,bin_total_bytes,bin_total_images)

if total_images==bin_total_images:
    print('CORRECT')
else:
    print('INCORRECT')
    

IOError: [Errno 2] No such file or directory: '/home/mario/datasets/KGS2016-train-labels-idx1-ubyte.gz'

## VIEW BOARDS IN SEQUENTIAL FILE

In [20]:
# explore bin file
from utils import Color
from board import Board
import utils

regs=[]
N=properties.boardsize
chunksize=N*N*properties.num_channels+2
with open(filename+'.bin') as f:
    for i in range(5):
        chunk=f.read(chunksize)
        if chunk:
            regs.append(chunk)
        else:
            break
board=Board(N)            
player_color=Color.WHITE
for line in regs:
    board,a=Board.get_board_and_move_from_register_str(N, line, player_color)
    print(a)
    board.play(player_color,utils.a2p(a,N))
    print(board)
    player_color=Board.opposite_color(player_color)
    #if player_color==Color.BLACK:
    #    player_color=Color.WHITE
    #else:
    #    player_color=Color.BLACK
    

111

Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . . . . . . . . . . . . . . 
18 . . . . . . . . . . . . . . . . . . . 
17 . . . . . . . . . . . . . . . . . . . 
16 . . . X . . . . . . . . . . . . . . . 
15 . . . . . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . .(O). . 
13 . . . . . . . . . . . . . . . . . . . 
12 . . . . . . . . . . . . . . . . . . . 
11 . . . . . . . . . . . . . . . . . . . 
10 . . . . . . . . . . . . . . . . . . . 
 9 . . . . . . . . . . . . . . . . . . . 
 8 . . . . . . . . . . . . . . . . . . . 
 7 . . . . . . . . . . . . . . . . . . . 
 6 . . . . . . . . . . . . . . . . . . . 
 5 . . . . . . . . . . . . . . . . . . . 
 4 . . . . . . . . . . . . . . . . . . . 
 3 . . . . . . . . . . . . . . . . . . . 
 2 . . . . . . . . . . . . . . . . . . . 
 1 . . . . . . . . . . . . . . . . . . . 
   A B C D E F G H J K L M N O P Q R S T 
149

Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . . . . . . . . . . . . . . 
18 . . .

## BOARDS IN SHUFFLED FILE


In [7]:
from utils import Color
from board import Board
import utils

regs=[]
N=properties.boardsize
chunksize=N*N*properties.num_channels+2
with open('shuffled_'+filename+'.bin') as f:
    for i in range(20):
        chunk=f.read(chunksize)
        if chunk:
            regs.append(chunk)
        else:
            break
board=Board(N)            
player_color=Color.WHITE
for line in regs:
    board,a=Board.get_board_and_move_from_register_str(N, line, player_color)
    print(a)
    board.play(player_color,utils.a2p(a,N))
    print(board)
    #if player_color==Color.BLACK:
    #    player_color=Color.WHITE
    #else:
    #    player_color=Color.BLACK

10

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 .(O)O X X X O . . 
 7 . . X O X O . . . 
 6 . . X O X O . . . 
 5 . X X O O O . . . 
 4 . X O O X X O O . 
 3 . O O . X . X O . 
 2 O X X O X X X X . 
 1 . O . . . . . . . 
   A B C D E F G H J 
20

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . X O O O X . . 
 7 . .(O)X O X . . . 
 6 . . O X O X . . . 
 5 . O O X X X . . . 
 4 . O X X O O X X . 
 3 . X X . O . O X . 
 2 X O O X O O O O . 
 1 . X . . . . . . . 
   A B C D E F G H J 
15

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . O X X X(O). . 
 7 . . . O X O . . . 
 6 . . X O X O . . . 
 5 . X X O O O . . . 
 4 . X O O X X O O . 
 3 . O O . X . X O . 
 2 O X X O X X X X . 
 1 . O . . . . . . . 
   A B C D E F G H J 
61

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . X O . . . 
 6 . . X . X O . . . 
 5 . X X O . O . . . 
 4 . X O O . .

## Experiments with seek()

In [18]:
from utils import Color
from board import Board
import numpy as np
import utils

num_examples=506
num_channels=4
boardsize=9
LABEL_SIZE=2
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'
dirname='9x9_10games'
pos_examples=np.array([i for i in range(num_examples)])
#np.random.shuffle(pos_examples)
chunksize = boardsize * boardsize * num_channels + LABEL_SIZE
with open(dirname+'_shuffled.bin','wb') as g:
    with open(dirname+'.bin','rb') as f:
        for i in [5,4,3,2,1,0]:
            f.seek(pos_examples[i]*chunksize)
            chunk = f.read(chunksize)
            board,a=Board.get_board_and_move_from_register_str(boardsize,chunk,Color.BLACK)
            p=utils.a2p(a,boardsize)
            cd=utils.p2cd(p,boardsize)
            print('a:{} move:{}'.format(a,cd))
            print(str(board))


a:60 move:G3

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . . O O . . 
 6 . . . . . . X . . 
 5 . . X . O . . . . 
 4 . . . . . . . . . 
 3 . . . . . . . . . 
 2 . . . . . . . . . 
 1 . . . . . . . . . 
   A B C D E F G H J 
a:24 move:G7

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . . X . . . 
 6 . . . . . . O . . 
 5 . . O . X . . . . 
 4 . . . . . . . . . 
 3 . . . . . . . . . 
 2 . . . . . . . . . 
 1 . . . . . . . . . 
   A B C D E F G H J 
a:38 move:C5

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . . O . . . 
 6 . . . . . . X . . 
 5 . . . . O . . . . 
 4 . . . . . . . . . 
 3 . . . . . . . . . 
 2 . . . . . . . . . 
 1 . . . . . . . . . 
   A B C D E F G H J 
a:23 move:F7

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . . . . . . 
 6 . . . . . . O .

## Explore individual shuffled files

In [21]:
from utils import Color
from board import Board
import numpy as np
import utils

num_examples=506
num_channels=4
boardsize=9
LABEL_SIZE=2
dirname='9x9_10games'
#pos_examples=np.array([i for i in range(num_examples)])
#np.random.shuffle(pos_examples)
chunksize = boardsize * boardsize * num_channels + LABEL_SIZE
with open(dirname+'_1.bin','rb') as g:    
    for i in [5,4,3,2,1,0]:
        f.seek(pos_examples[i]*chunksize)
        chunk = f.read(chunksize)
        board,a=Board.get_board_and_move_from_register_str(boardsize,chunk,Color.BLACK)
        p=utils.a2p(a,boardsize)
        cd=utils.p2cd(p,boardsize)
        print('a:{} move:{}'.format(a,cd))
        print(str(board))

    
    

NameError: name 'pos_examples' is not defined

## Explore 9x9_10games

In [56]:
from utils import Color,p2cd
from board import Board
import numpy as np
#from import utils

num_examples=506
num_channels=4
boardsize=9
LABEL_SIZE=2
chunksize = boardsize * boardsize * num_channels + LABEL_SIZE
#dirname='9x9_10games'
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'

#pos_examples=np.array([i for i in range(num_examples)])
#np.random.shuffle(pos_examples)
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'
#filename='/home/mario/datasets/KGS2016.bin'
filename='/home/mario/datasets/9x9_10games.bin'

with open(filename,'rb') as f:        
    #s=f.read()
    #print(s)
    for i in range(10):
        chunk = f.read(chunksize)
#        print(ord(chunk[0]))
#        #print(len(chunk))
        board,a=Board.get_board_and_move_from_register_str(boardsize,chunk,Color.BLACK)
       
        p=utils.a2p(a,boardsize)
        cd=p2cd(p,9)
        
        print(str(board))
        print('a:{} p:{} cd:{}'.format(a,p,cd))

    
    


Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . . . . . . 
 6 . . . . . . . . . 
 5 . . . . . . . . . 
 4 . . . . . . . . . 
 3 . . . . . . . . . 
 2 . . . . . . . . . 
 1 . . . . . . . . . 
   A B C D E F G H J 
a:40 p:55 cd:E5

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . . . . . . 
 6 . . . . . . . . . 
 5 . . . . O . . . . 
 4 . . . . . . . . . 
 3 . . . . . . . . . 
 2 . . . . . . . . . 
 1 . . . . . . . . . 
   A B C D E F G H J 
a:33 p:47 cd:G6

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . . . . . . 
 6 . . . . . . O . . 
 5 . . . . X . . . . 
 4 . . . . . . . . . 
 3 . . . . . . . . . 
 2 . . . . . . . . . 
 1 . . . . . . . . . 
   A B C D E F G H J 
a:23 p:36 cd:F7

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . . . . . 
 7 . . . . . O . . . 
 6 . . . . . . X . . 


##Explore KGS2016

In [7]:
from utils import Color,p2cd
from board import Board
import numpy as np
import utils

#num_examples=506
num_channels=4
boardsize=19
LABEL_SIZE=2
chunksize = boardsize * boardsize * num_channels + LABEL_SIZE
#dirname='9x9_10games'
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'

#pos_examples=np.array([i for i in range(num_examples)])
#np.random.shuffle(pos_examples)
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'
filename='/home/mario/datasets/KGS_10games.bin'
#filename='/home/mario/datasets/9x9_10games.bin'

with open(filename,'rb') as f:        
    #s=f.read()
    #print(s)
    for i in range(5):
        chunk = f.read(chunksize)
#        print(ord(chunk[0]))
#        #print(len(chunk))
        board,a=Board.get_board_and_move_from_register_str(boardsize,chunk,Color.BLACK)
       
        p=utils.a2p(a,boardsize)
        cd=p2cd(p,boardsize)
        
        print(str(board))
        print('a:{} p:{} cd:{}'.format(a,p,cd))



Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . . . . . . . . . . . . . . 
18 . . . . . . . . . . . . . . . . . . . 
17 . . . . . . . . . . . . . . . . . . . 
16 . . . . . . . . . . . . . . . O . . . 
15 . . . . . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . . . . . 
13 . . . . . . . . . . . . . . . . . . . 
12 . . . . . . . . . . . . . . . . . . . 
11 . . . . . . . . . . . . . . . . . . . 
10 . . . . . . . . . . . . . . . . . . . 
 9 . . . . . . . . . . . . . . . . . . . 
 8 . . . . . . . . . . . . . . . . . . . 
 7 . . . . . . . . . . . . . . . . . . . 
 6 . . . . . . . . . . . . . . . . . . . 
 5 . . . . . . . . . . . . . . . . . . . 
 4 . . . O . . . . . . . . . . . O . . . 
 3 . . . . . . . . . . . . . . . . . . . 
 2 . . . . . . . . . . . . . . . . . . . 
 1 . . . . . . . . . . . . . . . . . . . 
   A B C D E F G H J K L M N O P Q R S T 
a:78 p:103 cd:C15

Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . . . . . . . . . . . . . .

## Explore KGS2016

In [22]:
from utils import Color,p2cd
from board import Board
import numpy as np
np.set_printoptions(threshold='nan')
import utils

#num_examples=506
num_channels=4
boardsize=19
LABEL_SIZE=2

chunksize = boardsize * boardsize * num_channels + LABEL_SIZE
#dirname='9x9_10games'
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'

#pos_examples=np.array([i for i in range(num_examples)])
#np.random.shuffle(pos_examples)
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'
filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016_test.bin'
#filename='/home/mario/datasets/9x9_10games.bin'

with open(filename,'rb') as f:        
    #s=f.read()
    #print(s)
    for i in range(200):
        chunk = f.read(chunksize)
#        print(ord(chunk[0]))
#        #print(len(chunk))
        array_reg=np.array([ord(c) for c in chunk])[2:].reshape(num_channels,boardsize,boardsize)
        board,a=Board.get_board_and_move_from_register_str(boardsize,chunk,Color.BLACK)
       
        p=utils.a2p(a,boardsize)
        cd=p2cd(p,boardsize)
        
        print(str(board))
        print('a:{} p:{} cd:{}'.format(a,p,cd))
        #print(array_reg)
        


Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . . . . . . . . . . . . . . 
18 . . . . . . . . . . . . . . . . . . . 
17 . . . . . . . . . . . . . . . . . . . 
16 . . . O . . . . . . . . . . . . . . . 
15 . . . . . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . . . . . 
13 . . . . . . . . . . . . . . . . . . . 
12 . . . . . . . . . . . . . . . . . . . 
11 . . . . . . . . . . . . . . . . . . . 
10 . . . . . . . . . . . . . . . . . . . 
 9 . . . . . . . . . . . . . . . . . . . 
 8 . . . . . . . . . . . . . . . . . . . 
 7 . . . . . . . . . . . . . . . . . . . 
 6 . . . . . . . . . . . . . . . . . . . 
 5 . . . . . . . . . . . . . . . . . . . 
 4 . . . . . . . . . . . . . . . . . . . 
 3 . . . . . . . . . . . . . . . . . . . 
 2 . . . . . . . . . . . . . . . . . . . 
 1 . . . . . . . . . . . . . . . . . . . 
   A B C D E F G H J K L M N O P Q R S T 
a:111 p:137 cd:R14

Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . . . . . . . . . . . . . 

## Explore 9x9_10games

In [8]:
from utils import Color,p2cd
from board import Board
import numpy as np
np.set_printoptions(threshold='nan')
import utils

#num_examples=506
num_channels=4
boardsize=9
LABEL_SIZE=2

chunksize = boardsize * boardsize * num_channels + LABEL_SIZE
#dirname='9x9_10games'
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'

#pos_examples=np.array([i for i in range(num_examples)])
#np.random.shuffle(pos_examples)
#filename='/home/mario/datasets/KGS2016_dataset_ch4/KGS2016'
filename='/home/mario/datasets/9x9_10games_dataset/9x9_10games_5.bin'
#filename='/home/mario/datasets/9x9_10games.bin'

with open(filename,'rb') as f:        
    #s=f.read()
    #print(s)
    for i in range(200):
        chunk = f.read(chunksize)
        if not chunk:
            break
#        print(ord(chunk[0]))
#        #print(len(chunk))
        array_reg=np.array([ord(c) for c in chunk])[2:].reshape(num_channels,boardsize,boardsize)
        board,a=Board.get_board_and_move_from_register_str(boardsize,chunk,Color.BLACK)
       
        p=utils.a2p(a,boardsize)
        cd=p2cd(p,boardsize)
        
        print(str(board))
        print('a:{} p:{} cd:{}'.format(a,p,cd))
        #print(array_reg)
        


Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . O . . . . . . 
 7 . . X O . . X O . 
 6 . . O X X . O . . 
 5 . . O . . . . . . 
 4 . . . . X . O . . 
 3 . . X . . . . . . 
 2 . . . . . . . . . 
 1 . . . . . . . . . 
   A B C D E F G H J 
a:19 p:32 cd:B7

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . O . X . . . 
 8 . . . O X . X X . 
 7 . O . . O X X O . 
 6 O . O . O O O O . 
 5 . O X X X X X O . 
 4 X O O O X . O X O 
 3 . X O X O . X X . 
 2 X . X X O . . . . 
 1 . X . . . . . . . 
   A B C D E F G H J 
a:4 p:15 cd:E9

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . X O O . 
 8 . X . . . X X O . 
 7 X . . . X X O . . 
 6 O X X X . O O O . 
 5 O O O . O O X X . 
 4 . . . . X O X . . 
 3 . . . O O X X X . 
 2 . . . O . O X . . 
 1 . . . . O O O X . 
   A B C D E F G H J 
a:39 p:54 cd:D5

Black: 0 captures | White: 0 captures | Komi: 6.5
 9 . . . . . . . . . 
 8 . . . . . O O X . 
 7 . . . . O O X . . 
 6 . O O O . X . X . 
 